In [1]:
import math

import pandas as pd

from lol_fandom import get_tournaments
from lol_fandom import get_scoreboard_games


pd.set_option('display.max_columns', None)

In [2]:
class Team:
    """Team information"""
    q = math.log(10) / 400

    def __init__(self, name, league):
        self.name = name
        self.league = league
        self.win = 0
        self.loss = 0
        self.r = 1000
        self.RD = 350
        self.last_game_date = None

    def update_team_name(self, name):
        self.name = name

    def update_last_game_date(self, game_date):
        self.last_game_date = game_date

    @classmethod
    def get_g(cls, RDi):
        """Compute g(RDi)

        Args:
            RDi (float): Ratings Deviation (RD)

        Returns:
            float: g(RDi)
        """
        return 1 / math.sqrt(1 + (3 * cls.q ** 2 * RDi ** 2) / math.pi ** 2)

    @classmethod
    def get_e(cls, r0, ri, g):
        """Compute E(s|r0, ri, RDi)

        Args:
            r0 (float): previous rating
            ri (float): rating of opponent
            g (float): g(RDi)

        Returns:
            float: E(s | r0, ri, RDi)
        """
        return 1 / (1 + 10 ** ((g * (r0 - ri)) / -400))

    @classmethod
    def get_d(cls, g, e):
        """Compute d^2

        Args:
            g (float): g(RDi)
            e (float): E(s | r0, ri, RDi)

        Returns:
            float: d^2
        """
        return 1 / (cls.q ** 2 * g ** 2 * e * (1 - e))

    def init_rd(self):
        """Initialize RD"""
        self.RD = 350

    @classmethod
    def update_point(cls, team1, team2, result):
        """Update ratings of team1 and team2

        Args:
            team1 (Team): Team1
            team2 (Team): Team2
            result (int): 1 is Team1 win, 0 is Team2 win
        """
        assert isinstance(team1, Team)
        assert isinstance(team2, Team)

        team1_r = team1.r
        team2_r = team2.r
        team1_RD = team1.RD
        team2_RD = team2.RD

        team1._update_point(team2_r, team2_RD, result)
        team2._update_point(team1_r, team1_RD, 1 - result)

    def _update_point(self, ri, RDi, s):
        if s == 1:
            self.win += 1
        else:
            self.loss += 1

        g_RD = Team.get_g(RDi)
        e = Team.get_e(self.r, ri, g_RD)
        d_2 = Team.get_d(g_RD, e)
        self.r = self.r + Team.q / (1 / self.RD ** 2 + 1 / d_2) * g_RD * (s - e)

        self.RD = math.sqrt((1 / self.RD ** 2 + 1 / d_2) ** -1)

    def get_win_prob(self, opponent):
        """Get win probability

        Args:
            opponent (Team): Opponent team

        Returns:
            float: Win probability
        """
        return Team.get_e(self.r, opponent.r, Team.get_g(opponent.RD))

    def to_dict(self):
        """Make Team class to dictionary

        Returns:
            dict: Dictionary of Team instance
        """
        data = {
            'Team': self.name,
            'League': self.league,
            'Win': self.win,
            'Loss': self.loss,
            'WinRate': self.win / (self.win + self.loss) if self.win != 0 else 0,
            'r': self.r,
            'RD': self.RD,
            'last_game_date': self.last_game_date,
        }

        return data

def get_team_id(teams_id, name):
    return teams_id.loc[teams_id['team'] == name, 'team_id'].iloc[0]

def proceed_rating(teams_id, teams, games):
    """Proceed rating with teams and games

    Args:
        teams_id (DataFrame): ID of teams 
        teams (list): List of Team instances
        games (DataFrame): Scoreboard games
    """
    for row in games.itertuples():
        team1, team2 = row.Team1, row.Team2
        game_date = row._6
        result = 1 if row.WinTeam == team1 else 0
        id1, id2 = get_team_id(teams_id, team1), get_team_id(teams_id, team2)
        Team.update_point(teams[id1], teams[id2], result)
        teams[id1].update_last_game_date(game_date)
        teams[id2].update_last_game_date(game_date)

def get_rating(teams):
    """Get rating of teams

    Args:
        teams (list): List of Team instances

    Returns:
        DataFrame: Rating of teams
    """
    ratings = pd.DataFrame(data=map(lambda x: x.to_dict(), teams.values()))
    ratings = ratings.sort_values(by='r', ascending=False).reset_index(drop=True)
    return ratings

def get_team_name(same_team_names, name):
    """Get latest name of the team

    Args:
        same_team_names (dict): Dictionary of names of same teams
        name (str): Name of the team

    Returns:
        str: Latest name of the team
    """
    while name in same_team_names:
        name = same_team_names[name]
    return name

In [3]:
TARGET_LEAGUES = [
    'LCK', 'LPL', 'LoL EMEA Championship', 'EU LCS', 'LCS', 'NA LCS',
    'MSI', 'WCS',
    'PCS', 'VCS', 'LJL', 'CBLOL', 'LLA', 'LCO', 'TCL', 'LCL',
]

teams_id = pd.read_csv('teams_id.csv')
teams_id

,team,team_id
0,100 Thieves,253
1,1UP Gaming,202
2,2144 Danmu Gaming,91
3,5 Ronin,386
4,7th heaven,294
...,...,...
527,mousesports,123
528,n!faculty,168
529,paiN Gaming,262
530,team propertybb2,139


In [4]:
teams = {}
for year in range(2022, 2023):
    tournaments = pd.DataFrame()
    for league in TARGET_LEAGUES:
        t = get_tournaments(where=f'L.League_Short="{league}" and T.Year={year}')
        tournaments = pd.concat([tournaments, t])
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    for page in tournaments['OverviewPage']:
        print(f'{page} rating ...')
        scoreboard_games = get_scoreboard_games(f'T.OverviewPage="{page}"')
        if scoreboard_games is None:
            print(f'{page} is None\n')
            continue
        print(f'{scoreboard_games.shape[0]} games')
        scoreboard_games = scoreboard_games.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)

        league = page.split('/')[0]
        team_names = scoreboard_games[['Team1', 'Team2']].unstack().unique()
        team_check = True
        for name in team_names:
            if name not in teams_id['team'].values:
                print(f'{name} not in teams')
                team_check = False
                break
            id = get_team_id(teams_id, name)
            if id not in teams:
                teams[id] = Team(name, league)
            else:
                teams[id].update_team_name(name)
        if not team_check:
            break
        
        for name in team_names:
            id = get_team_id(teams_id, name)
            teams[id].init_rd()

        proceed_rating(teams_id, teams, scoreboard_games)

    if not team_check:
        break
if team_check:
    rating = get_rating(teams)

LLA/2022 Season/Opening Promotion rating ...
25 games
LPL/2022 Season/Spring Season rating ...
341 games
LCK/2022 Season/Spring Season rating ...
212 games
LCS/2022 Season/Lock In rating ...
39 games
LEC/2022 Season/Spring Season rating ...
90 games
CBLOL/2022 Season/Split 1 rating ...
90 games
TCL/2022 Season/Winter Season rating ...
91 games
LCO/2022 Season/Split 1 rating ...
84 games
LLA/2022 Season/Opening Season rating ...
58 games
LCS/2022 Season/Spring Season rating ...
92 games
PCS/2022 Season/Spring Season rating ...
91 games
LJL/2022 Season/Spring Season rating ...
84 games
VCS/2022 Season/Spring Season rating ...
132 games
LCL/2022 Season/Spring Season rating ...
16 games
LLA/2022 Season/Opening Playoffs rating ...
32 games
LCK/2022 Season/Spring Playoffs rating ...
20 games
LEC/2022 Season/Spring Playoffs rating ...
30 games
CBLOL/2022 Season/Split 1 Playoffs rating ...
32 games
TCL/2022 Season/Winter Playoffs rating ...
20 games
LPL/2022 Season/Spring Playoffs rating ...
5

In [5]:
rating

,Team,League,Win,Loss,WinRate,r,RD,last_game_date
0,DRX,LCK,72,59,0.549618,1784.208183,89.080365,2022-11-06 04:42:00
1,T1,LCK,107,37,0.743056,1759.161765,94.177055,2022-11-06 04:42:00
2,JD Gaming,LPL,81,40,0.669421,1688.851244,124.560559,2022-10-30 00:10:00
3,Gen.G,LCK,83,29,0.741071,1667.665125,106.898193,2022-10-31 00:03:00
4,DWG KIA,LCK,71,46,0.606838,1665.451836,127.555479,2022-10-23 00:49:00
...,...,...,...,...,...,...,...,...
118,Dewish Team,PCS,2,16,0.111111,518.186146,148.196863,2022-08-06 09:22:00
119,Rensga Esports,CBLOL,7,29,0.194444,512.505134,134.124322,2022-08-07 18:54:00
120,SEM9,PCS,2,34,0.055556,489.096355,174.084668,2022-08-06 09:22:00
121,5 Ronin,TCL,13,29,0.309524,400.831951,146.708141,2022-08-14 13:15:00
